In [3]:
pip install py2neo

Note: you may need to restart the kernel to use updated packages.


In [29]:
from py2neo import Graph

graph = Graph("bolt://localhost:7687", auth=("neo4j", "12345678"))

def generate_description(hid):
    query = f"""
    MATCH (h:Property {{hid: '{hid}'}})
    OPTIONAL MATCH (h)-[:NEAR_STORE]->(s:Store)
    RETURN h, collect(s) as stores
    """
    result = graph.run(query).data()

    if not result:
        return "未找到與該HID相關的房屋資料。"

    house_info = result[0]['h']
    stores = result[0]['stores']
    
    descriptions = []

    if house_info.get("address"):
        descriptions.append(f"房屋地址位於{house_info['address']}。")
    if house_info.get("type"):
        descriptions.append(f"該房屋類型為{house_info['type']}。")
    if house_info.get("pattern"):
        descriptions.append(f"房屋格局為{house_info['pattern']}。")
    if house_info.get("size"):
        descriptions.append(f"房屋大小為{house_info['size']}坪。")
    if house_info.get("layer"):
        descriptions.append(f"樓層為{house_info['layer']}。")
    if house_info.get("price"):
        descriptions.append(f"房屋租金為{house_info['price']}元。")
    if house_info.get("subway"):
        descriptions.append(f"距離最近的地鐵站為{house_info['subway']}。")
    if house_info.get("bus"):
        descriptions.append(f"附近的公車站包括{house_info['bus']}。")

    if stores:
        store_names = [store.get('name') for store in stores if store.get('name')]
        if store_names:
            descriptions.append(f"附近有以下店家：{', '.join(store_names)}。")

    return " ".join(descriptions)

hid = input("請輸入HID：")
description = generate_description(hid)
print(description)


請輸入HID： 16343386


房屋地址位於士林區重慶北路四段。 該房屋類型為公寓。 房屋格局為2房1廳1衛。 房屋大小為24坪。 樓層為5F/5F。 房屋租金為19999元。 距離最近的地鐵站為距劍潭站1394公尺。 附近的公車站包括距社子消防分隊站539公尺, 距社子派出所站765公尺。 附近有以下店家：醫院, 便利商店, 百貨公司, 夜市, 公園, 傳統市場。
